In [1]:
import requests as req
from datetime import datetime
import pandas as pd

from bs4 import BeautifulSoup as BS
from nltk.sentiment.vader import SentimentIntensityAnalyzer 

from connection.var import ConnectionVar_forex, ConnectionVar_crypto, agent_desktop, parser, URL

In [3]:
crypto_columns = ['DATE','PRICE','PRICE_CAP', 'VOLUME_24','CIRCULATION', 'PERCENT_CHG']
data_columns = ['NAME','PRICE','PRICE_CAP', 'VOLUME_24','CIRCULATION', 'PERCENT_CHG']
tag_names = 'a'
forex_columns = ['DATE','LAST','CHG%','CHG','BID','ASK','HIGH','LOW','RaTING']

SIA = SentimentIntensityAnalyzer()

def get_html(url, header):
    #get data from sources
    #url for various sources
    connect = req.get(url, header)
    data = connect.content
    return data

def scrape_data_from_web(where, url_id, tag=None, className=None, id_name=None):

    date_as = datetime.strftime(datetime.now(), '%m-%d-%Y, %H:%M')
    scraped_data = []
    
    if where == 'crypto':
        
        conn = get_html(URL[url_id], header=agent_desktop)
        soup = BS(conn, parser)
        
        for k,v in ConnectionVar_crypto.items():
            scraped_data.append([x.get_text() for x in soup.find_all(tag, class_=v)])
        
        df = pd.DataFrame(index=scraped_data[0], columns=crypto_columns)

        df['DATE'] = date_as
        df['PRICE'] = scraped_data[1]
        df['PRICE_CAP'] = scraped_data[2]
        df['VOLUME_24'] = scraped_data[3]
        df['CIRCULATION'] = scraped_data[4]
        df['PERCENT_CHG'] = scraped_data[5]

        #df.to_sql(sqlite_table, sqlite_conn, if_exists='append')
        return df.head(10)
    
    elif where == 'forex':
        
        conn = get_html(URL[url_id], header=agent_desktop)
        soup = BS(conn, parser)
        
        
        scraped_data.append(([x.get_text() for x in soup.find_all('a',{'class':ConnectionVar_forex['a']})]))
        scraped_data.append([x.get_text() for x in soup.find_all('td',{'class':ConnectionVar_forex['td']})])
            
        #df = pd.DataFrame(index=scraped_data[0], columns=forex_columns)
        #df['DATE'] = date_as
        #df['LAST'] = scraped_data[1]
        #df['CHG%'] = scraped_data[2]
        #df['CHG'] = scraped_data[3]
        #df['BID'] = scraped_data[4]
        #df['ASK'] = scraped_data[5]
        #df['HIGH'] = scraped_data[6]
        #df['LOW'] = scraped_data[7]
        #df['RaTING'] = scraped_data[8]
        print(len(scraped_data[0]))
        print(len(scraped_data[1]))
        print(scraped_data)
        #df.to_sql(sqlite_table, sqlite_conn, if_exists='append')
        #return df.head(10)
        
    elif where == 'news'.lower():
        conn = get_html(URL[url_id], header=agent_desktop)
        soup = BS(conn, parser)
        
        scraped_data.append(([x.get_text() for x in soup.find_all(tag,class_=className)]))

        return scraped_data
        


In [8]:
newsHeadlines = scrape_data_from_web(where='news', tag='div', className='o-teaser__heading', url_id=3)
newsArticles = scrape_data_from_web(where='news', tag='p', className='o-teaser__standfirst', url_id=3)

In [9]:
newsHeadlinesPolarity = []

for polarityScores in range(len(newsHeadlines[0])):
    newsHeadlinesPolarity.append(SIA.polarity_scores(newsHeadlines[0][polarityScores]))   

In [10]:
newsDataFrame = pd.DataFrame(newsHeadlinesPolarity, columns=['neg','neu','pos'])

In [23]:
newsDataFrame['newsHeadline'] = newsHeadlines[0]


In [28]:
cryptoData = scrape_data_from_web(where='crypto')

'Business banking hopes to ride wave of new start-ups Premium'

In [32]:
for x in range(len(newsDataFrame)):
    reqSend = req.post("http://127.0.0.1:5050/postnews/" + str(x) , data = {
        "newsArticle":newsDataFrame['newsHeadline'][x],
        "newsPolarityNeg":newsDataFrame['neg'][x],
        "newsPolarityPos":newsDataFrame['pos'][x],
        "newsPolarityNeu":newsDataFrame['neu'][x]

    })